In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from functools import *

dataset = pd.read_csv('../mnist/mnist_train.csv')
testset = pd.read_csv('../mnist/mnist_test.csv')

In [28]:
class Perceptron(object):
    def __init__(self,input_size, number=-1, lr=1, epochs=100):
        self.W = np.random.normal(0, 0.5, input_size+1)
        self.epochs = epochs
        self.lr = lr
        self.number = number
    
    def activation_fn(self, x):
        return (1 / (1 + np.exp(-x)))
 
    def predict(self, x):
        z = self.W.T.dot(x)
        a = self.activation_fn(z)
        return a
    
    def guess_new(self, y):
        x = np.insert(y, 0, 1)
        z = self.W.T.dot(x)
        a = self.activation_fn(z)
        return a
 
    def fit(self, X, d):
        for _ in range(self.epochs):
            for i in range(d.shape[0]):
                x = np.insert(X[i], 0, 1)
                y = self.predict(x)
                e = d[i] - y
                self.W = self.W + self.lr * e * x

In [64]:
x_train = np.array(dataset)[0:100, 1:] / 255
y_train = np.array(dataset)[0:100, 0]

x_test = np.array(testset)[:, 1:] / 255
y_test = np.array(testset)[:, 0]

In [71]:
#Train perceptron by correspondent number

perseptrons = [Perceptron(input_size=x_train.shape[1], lr=0.1, epochs=10,number=i) for i in range(10)]

for perceptron in perseptrons:
    perceptron.fit(x_train, y_train)
    #Get Accuracy
    total = y_test.shape[0]
    err_total = 0
    acc_total = 0
    for test_case, test_answer in zip(x_test, y_test):
        prediction = round(perceptron.guess_new(test_case))
        if (prediction == 1 and perceptron.number == test_answer):
            acc_total +=1
        else:
            err_total +=1
    print('Accuracy of Perceptron number ' + str(perceptron.number) + ' is: ' + str(float(acc_total)/float(total))[:4])

Accuracy of Perceptron number 0 is: 0.09
Accuracy of Perceptron number 1 is: 0.11
Accuracy of Perceptron number 2 is: 0.10
Accuracy of Perceptron number 3 is: 0.10
Accuracy of Perceptron number 4 is: 0.09
Accuracy of Perceptron number 5 is: 0.08
Accuracy of Perceptron number 6 is: 0.09
Accuracy of Perceptron number 7 is: 0.10
Accuracy of Perceptron number 8 is: 0.09
Accuracy of Perceptron number 9 is: 0.10
